In [10]:
# Importamos las librerias a usar
import pandas as pd
import json
import ast

In [11]:
#Convertir el archivo a Json
dataSteamGames = 'output_steam_games.json'
columnasTipo = {'steam_id': str}
dfSteamGames= pd.read_json(dataSteamGames, lines = True, dtype = columnasTipo)

In [12]:
#Informacion general del dataframe
filas,columnas = dfSteamGames.shape
print("El numero de filas : ",filas)
print("El numero de columnas : ",columnas,'\n')
print("Informacion general de la tabla dfSteamGames\n")
print(dfSteamGames.info())

El numero de filas :  120445
El numero de columnas :  19 

Informacion general de la tabla dfSteamGames

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 19 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   publisher       24083 non-null   object 
 1   genres          28852 non-null   object 
 2   app_name        32133 non-null   object 
 3   title           30085 non-null   object 
 4   url             32135 non-null   object 
 5   release_date    30068 non-null   object 
 6   tags            31972 non-null   object 
 7   reviews_url     32133 non-null   object 
 8   discount_price  225 non-null     float64
 9   specs           31465 non-null   object 
 10  price           30758 non-null   object 
 11  early_access    32135 non-null   float64
 12  id              32133 non-null   float64
 13  metascore       2677 non-null    object 
 14  developer       28836 non-null   object 
 1

In [13]:
#Eliminar las columnas menos relevantes del Dataframe
columnasAEliminar = ['url','tags','reviews_url','discount_price','specs', 'price','developer','items','steam_id','items_count','publisher','app_name','metascore','early_access']
dfSteamGames = dfSteamGames.drop(columns=columnasAEliminar)
dfSteamGames

,genres,title,release_date,id,user_id
0,None,None,None,NaN,76561197970982479
1,None,None,None,NaN,js41637
2,None,None,None,NaN,evcentric
3,None,None,None,NaN,Riot-Punch
4,None,None,None,NaN,doctr
...,...,...,...,...,...
120440,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,2018-01-04,773640.0,None
120441,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,2018-01-04,733530.0,None
120442,"[Indie, Racing, Simulation]",Russian Roads,2018-01-04,610660.0,None
120443,"[Casual, Indie]",EXIT 2 - Directions,2017-09-02,658870.0,None


In [14]:
# Conversiones
# 1 Columna Genres
# Reemplazar con listas vacias la columnas genres antes de explotarlos y soltar filas con muchos valores vacios
def reemplazarGenres(valor):
    if valor is None:
        return []
    else:
        return valor
# Aplicar la función a todo el DataFrame usando applymap
dfSteamGames['genres'] = dfSteamGames['genres'].apply(reemplazarGenres)

# 2 Explorar la columna generes
dfSteamGames = dfSteamGames.explode('genres')

# A toda las filas
#  Sacar las filas que tienen al menos 2 valores no Nulos
dfSteamGames= dfSteamGames.dropna(thresh=2)

#Convertir columnas release date, tomando en cuenta como NAN las fechas sin formato datetime
dfSteamGames['release_date'] = pd.to_datetime(dfSteamGames['release_date'],errors='coerce')

# 2 Eliminar las filas que no tengan genres y release_date como dato
dfSteamGames = dfSteamGames.dropna(subset=['genres', 'release_date'])

#Convertir columnas id a tipo entero tomando en cuenta Nan y luego pasarlas a string
dfSteamGames['id'] = pd.to_numeric(dfSteamGames['id'], errors='coerce').fillna(0).astype(int)
dfSteamGames['id'] = dfSteamGames['id'].astype(str)

#Crear una columna años 
dfSteamGames['years'] = dfSteamGames['release_date'].dt.year
#Reseteamos los indices 
dfSteamGames = dfSteamGames.reset_index(drop=True)
#Depues de verificacion, eliminar nulos de title
dfSteamGames = dfSteamGames.dropna(subset=['title'])
dfSteamGames

,genres,title,release_date,id,user_id,years
0,Action,Lost Summoner Kitty,2018-01-04,761140,None,2018
1,Casual,Lost Summoner Kitty,2018-01-04,761140,None,2018
2,Indie,Lost Summoner Kitty,2018-01-04,761140,None,2018
3,Simulation,Lost Summoner Kitty,2018-01-04,761140,None,2018
4,Strategy,Lost Summoner Kitty,2018-01-04,761140,None,2018
...,...,...,...,...,...,...
70699,Indie,Russian Roads,2018-01-04,610660,None,2018
70700,Racing,Russian Roads,2018-01-04,610660,None,2018
70701,Simulation,Russian Roads,2018-01-04,610660,None,2018
70702,Casual,EXIT 2 - Directions,2017-09-02,658870,None,2017


In [15]:
dfSteamGames.isna().sum()

genres              0
title               0
release_date        0
id                  0
user_id         70702
years               0
dtype: int64

In [16]:
# Conversiones generales extras
#Eliminar despues de verificacion
columnasAEliminar = ['user_id']
dfSteamGames = dfSteamGames.drop(columns=columnasAEliminar)

#Renombrar las columnas para poder hacer el Merge
dfSteamGames = dfSteamGames.rename(columns={'id': 'item_id'})
dfSteamGames

,genres,title,release_date,item_id,years
0,Action,Lost Summoner Kitty,2018-01-04,761140,2018
1,Casual,Lost Summoner Kitty,2018-01-04,761140,2018
2,Indie,Lost Summoner Kitty,2018-01-04,761140,2018
3,Simulation,Lost Summoner Kitty,2018-01-04,761140,2018
4,Strategy,Lost Summoner Kitty,2018-01-04,761140,2018
...,...,...,...,...,...
70699,Indie,Russian Roads,2018-01-04,610660,2018
70700,Racing,Russian Roads,2018-01-04,610660,2018
70701,Simulation,Russian Roads,2018-01-04,610660,2018
70702,Casual,EXIT 2 - Directions,2017-09-02,658870,2017


In [17]:
dfSteamGames.to_csv('steamGamesLimpio.csv', index=False)